<a href="https://colab.research.google.com/github/brian-ho/intro-to-urban-data/blob/main/Measure/M08_Working_with_Rhino_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **08 Measure** Working with Rhino Files
```
    Class       ARCH 6131 Urban Design Methods / Skills / Tools 1
                Tata Innovation Center
                Fall 2025
                Monday 9:30 AM to 12 PM

    Instructor  Brian Ho
                brian@brian-ho.io
```

This notebook shows how you can read and write Rhinoceros 3DM files _programmatically_. This does not replace your use of Rhino as a desktop app: you will always need to work more directly as a designer. But being able to open and edit Rhino files is a powerful way to connect your data to your choice of drawing and design.

This notebook covers:
- Reading a Rhino file
- Accessing layers and geometry
- Creating layers and geometry
- Saving a Rhino file

Almost all the functionality in this notebook depends on a Python library from McNeel called  `rhino3dm`. This exposes functionality to read and write all geometry types, and is based on an open standard. You can learn more about it [here](https://github.com/mcneel/rhino3dm).

In [4]:
!pip install rhino3dm --quiet

In [3]:
# Add McNeel's Python library
import rhino3dm

Connect to drive to access your custom files.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
EXAMPLE_DATA_DIRECTORY = "example_data"

## Reading a Rhino file
Reading a Rhino file with `rhino3dm` is very easy! Much like reading a CSV or GeoJSON, you can use a helpful `Read()` method on a `File3dm` class.

Note that `rhino3dm`'s convention is to use [camel case or Pascal case](https://en.wikipedia.org/wiki/Camel_case) for naming. This means consecutive words each have a first uppercase letter without spaces, like `NurbsCurveKnotList`. It's more common to see [snake case](https://en.wikipedia.org/wiki/Snake_case) in Python. This means consecutive words are separated by an underscore and are all lowercase, which looks something like: `read_csv`. You'll need to be sure to capitalize correctly when using `rhino3dm`.

In [ ]:
# Let's read a 3DM file
# File3dm.Read() takes a string path, so you'll need to make sure your path is created correctly
rhino_file_path = EXAMPLE_DATA_DIRECTORY + "/example_rhino_file.3dm"

print(
    f"Reading Rhino file from: '{EXAMPLE_DATA_DIRECTORY + '/example_rhino_file.3dm'}'"
)
example_3dm = rhino3dm.File3dm.Read(rhino_file_path)

if example_3dm is None:
    print(
        f"Unable to read Rhino file at '{EXAMPLE_DATA_DIRECTORY + '/example_rhino_file.3dm'}'! Did you use the right path?"
    )

example_3dm

Reading Rhino file from: 'example_data/example_rhino_file.3dm'


If you run into errors here, check your `rhino_file_path`. The `File3dm.Read()` method takes only a string as its argument for where to look for the file, rather than a smarter `Path` object. This means you have to make sure the full path is correctly written with a forward slash ("/") between directories.

There are some useful attributes on the `File3dm` class — bits of information an object. You can read about them [here](https://mcneel.github.io/rhino3dm/python/api/File3dm.html). As you work with `rhino3dm` and other Python libraries, reading the docs will always be a good way to learn.

## Accessing layers and geometry
So we've opened the file. What now? Usefully, we can take a look at more details about the data within the Rhino file. Let's try this with layers and geometry — the two essential building blocks for working with Rhino flies.

Layers are stored on a `File3dmLayerTable` object attached to each `File3dm`. You can access it with the `Layer` property.

In [ ]:
# Let's look at the layers property on our Rhino file.
example_3dm.Layers

`Layers` returns an iterable object. By itself, this isn't that useful to look at — it's a container for more data that we want.

Since the `File3dmLayerTable` is an iterable object, we can use a simple `for` loop to check out its contents. This will give us a single `Layer` object for each loop of the iteration. We can then access more meaningful attributes and properties on the layer.

I've printed out some below. Check out others in the [docs](https://mcneel.github.io/rhino3dm/python/api/Layer.html).

In [ ]:
# Let's try iterating over the layers
print(f"There are {len(example_3dm.Layers)} layers in the Rhino file.\n")

# For each layer, you can get more information
for example_layer in example_3dm.Layers:
    print(f"Layer name: '{example_layer.Name}'")
    print(f"Full path: '{example_layer.FullPath}'")
    print(f"index: {example_layer.Index}")
    print(f"Visible: {example_layer.Visible}")
    print(f"Locked: {example_layer.Locked}")
    print(f"Color: {example_layer.Color}\n")

There are 5 layers in the Rhino file.

Layer name: 'Default'
Full path: 'Default'
index: 0
Visible: True
Locked: False
Color: (0, 0, 0, 255)

Layer name: '3D Geometry'
Full path: '3D Geometry'
index: 1
Visible: True
Locked: False
Color: (0, 0, 0, 255)

Layer name: 'Volumes'
Full path: '3D Geometry::Volumes'
index: 2
Visible: True
Locked: False
Color: (200, 0, 0, 255)

Layer name: 'Surfaces'
Full path: '3D Geometry::Surfaces'
index: 3
Visible: True
Locked: False
Color: (125, 38, 205, 255)

Layer name: 'Curves'
Full path: 'Curves'
index: 4
Visible: True
Locked: False
Color: (0, 0, 255, 255)



Try opening up the `example_rhino_file.3dm` file directly in Rhino and comparing it to the above. A few observations:

- Nested Rhino layers have a "::" separator in them between child and parent layers. So "Layer A::Layer B" means Layer B is the child layer of Layer A
- Each layer has a unique index
- Layers have `bool` values (short for boolean) for their visible and locked states: e.g. they can be either `True` or `False`.
- Layer colors are stored as tuples of RGBA type (look familiar?)

OK, now let's try to do the same with geometry. `File3dm.Objects` will give us `File3dmObjectTable`.

In [ ]:
# Here's the object table
example_3dm.Objects

As, before we can iterate over this object as if it were a list. Note that each `File3dmObject` has two main properties: `Attributes` and `Geometry`.

Each stores related informatoin about the object.

In [ ]:
# Let's try iterating over the geometry
print(f"There are {len(example_3dm.Objects)} objects in the Rhino file.\n")

for example_object in example_3dm.Objects:
    # For simplicity, let's use variable to reference the attributes and geometry
    attributes = example_object.Attributes
    geometry = example_object.Geometry

    print(f"Geometry type: {geometry.ObjectType}")
    print(f"ID: '{attributes.Id}'")
    # We can get the name of the layer using the layer's index
    print(f"Layer: '{example_3dm.Layers.FindIndex(attributes.LayerIndex).Name}'\n")

There are 7 objects in the Rhino file.

Geometry type: ObjectType.Extrusion
ID: 'c0c3e292-2bbd-460e-9435-44528602fc4c'
Layer: 'Volumes'

Geometry type: ObjectType.Curve
ID: '6b3a5b45-af08-4cbe-9061-a01d2a48ff8e'
Layer: 'Curves'

Geometry type: ObjectType.Brep
ID: '1211cc07-8074-4a13-99e6-038c139d3198'
Layer: 'Surfaces'

Geometry type: ObjectType.Brep
ID: '5c417cf2-b825-45ae-ba4d-5aa8f7697b40'
Layer: 'Volumes'

Geometry type: ObjectType.Curve
ID: '3b83f854-68be-4268-8cbf-e897fa04f9b2'
Layer: 'Curves'

Geometry type: ObjectType.Curve
ID: '9ede57be-3b5d-4ebd-ae6d-cbf61a330f9a'
Layer: 'Curves'

Geometry type: ObjectType.Brep
ID: 'aa10934a-ad97-4af1-b0fa-6b40e38ac23b'
Layer: 'Surfaces'



Pretty cool, right? So we can see that each object has a type, an ID and we can find its layer through the layer index.

## Creating layers and geometry
Now that we know how to read a Rhino file's layers and geometry, let's try it in the reverse. Can we write layers and geometry to a Rhino file? You be we can!

Let's start by creating a new Rhino, empty file.

In [ ]:
# Make an empty file
new_3dm = rhino3dm.File3dm()

Just like you would using the Rhino app on your desktop, you should check and set the unit system.

In [ ]:
# What's the unit system?
print(f"The file's units are in {new_3dm.Settings.ModelUnitSystem}")

# How many layers are in the empty file?
print(f"There are {len(new_3dm.Layers)} layers in the file")

The file's units are in UnitSystem.Millimeters
There are 0 layers in the file


Setting the unit system is just a matter of assigning to the attribute! What you see below may be somewhat confusing: we're using a special `rhino3dm` class to represent the unit system: `UnitSystem`. This class is known as an enumeration — it can represent one of a few specific, special meanings.

Internally, these are often represented as `int`s for efficiency. Enumeration lets you assign specific names to each `int`. In this case, `UnitSystem.Meters` has the value 4. Note that the `int` representation doesn't have to map to the meaning. An enumeration could represent multiple discrete meanings that have no order nor relative relationship.

While the use of an enumeration here is just a detail, in the future we will return to this concept when we discuss "categorical" data.

In [ ]:
# Let's set the units appropriately
new_3dm.Settings.ModelUnitSystem = rhino3dm.UnitSystem.Meters

print(
    f"The unit system is set to {new_3dm.Settings.ModelUnitSystem} which has the value {new_3dm.Settings.ModelUnitSystem.value}"
)

The unit system is set to UnitSystem.Meters which has the value 4


Now, let's try adding a layer. There's a handy `File3dmLayerTable.AddLayer()` method, which takes two arguments: a string that names the layer, and a color (as an RGBA tuple).

Let's create a simple layer. Note that 3DM files supports adding multiple layers with the same name. Much like when you use Rhino, the layers simply get a suffix.

In [ ]:
# Let's set a "constant" variable for red
RED_RGBA = (255, 0, 0, 255)

# Add the layer with a name and color
new_3dm.Layers.AddLayer("My First Layer", RED_RGBA)

0

In [ ]:
# Check the results
print(f"There are {len(new_3dm.Layers)} layers in the file")
print(f"Layers are: {[f'{l.FullPath} at index {l.Index}' for l in new_3dm.Layers]}")

There are 1 layers in the file
Layers are: ['My First Layer at index 0']


That's all it takes to add layer!

Layers are more straightforward than geometry, unfortunately. Geometry iis added to the `File3dmObjectTable`, but doing so requires you define both a geometry primitive that is one of a `BaseGeometry` type, as well as a set of attributes for things like the layer of the geometry.

Let's start with a simple `Point`. You need to construct a `Point` object using one argument: its 3D location, which is represented by a `Point3d` object.

In [ ]:
# Point
# First, create a point geometry from a 3D location
test_pt = rhino3dm.Point(rhino3dm.Point3d(0, 0, 0))

⚠ **Point and Point3d are not the same, and you can only add Point objects!**

_You can only add geometry that is derived from `BaseGeometry`. The `Point` class is one such example. The `Point3d` class, on the other hand, is used to represent locations in space, but is not itself a geometry._


Next, we can set up attributes. As you might know from using Rhino on your computer, there are many attributes you can set on a geometry object. In our case, we just want to set the layer. But take a look at the [docs](https://mcneel.github.io/rhino3dm/python/api/ObjectAttributes.html) and see if there are other pieces of information you want to set.

We do this by first creating a `ObjectAttributes` object, then settinig its `LayerIndex` attribute. "0" is the index for the single layer we added previously.

In [ ]:
# Let's define geometry attributes that we can re-use
test_attributes = rhino3dm.ObjectAttributes()

# This sets the attributes so that the objects is on the layer with index 0
test_attributes.LayerIndex = 0

OK, now that we have both geometry and attributes, we can add them to the Rhino file! If this works, the `File3dmObjectTable.Add()` method will return the Universally Unique Identified (UUID) for the geometry.

In [ ]:
# Add the geometry and object to the Rhino file
new_3dm.Objects.Add(test_pt, test_attributes)

UUID('fbc033e5-f3b3-4746-8b47-a6f8222f1891')

... and we can also check to see that the object table has been incremented.

In [ ]:
print(f"There are {len(new_3dm.Objects)} objects in the Rhino file.")

There are 1 objects in the Rhino file.


So that's how you add a `Point`. What about other geometry?

Again, there's some nuance. You can only add geometry that is derived from `GeometryBase`. For reference, I've printed them out below.

In [ ]:
def get_subclasses(cls):
    """A helper to recursively get the subclasses for a given base class.

    Uses a set to de-dupe in the case of multiple inheritance.
    """
    return set(cls.__subclasses__()).union(
        [sc for cl in cls.__subclasses__() for sc in get_subclasses(cl)]
    )


base_geometry_types = get_subclasses(rhino3dm.GeometryBase)

print("BaseGeometry types in rhino3dm are:")
for bg in base_geometry_types:
    print(
        "- " + bg.__name__,
        "https://mcneel.github.io/rhino3dm/python/api/" + bg.__name__,
    )

BaseGeometry types in rhino3dm are:
- BrepFace https://mcneel.github.io/rhino3dm/python/api/BrepFace
- CurveProxy https://mcneel.github.io/rhino3dm/python/api/CurveProxy
- PolylineCurve https://mcneel.github.io/rhino3dm/python/api/PolylineCurve
- Brep https://mcneel.github.io/rhino3dm/python/api/Brep
- InstanceReference https://mcneel.github.io/rhino3dm/python/api/InstanceReference
- PointGrid https://mcneel.github.io/rhino3dm/python/api/PointGrid
- PolyCurve https://mcneel.github.io/rhino3dm/python/api/PolyCurve
- Curve https://mcneel.github.io/rhino3dm/python/api/Curve
- Surface https://mcneel.github.io/rhino3dm/python/api/Surface
- ArcCurve https://mcneel.github.io/rhino3dm/python/api/ArcCurve
- SurfaceProxy https://mcneel.github.io/rhino3dm/python/api/SurfaceProxy
- Hatch https://mcneel.github.io/rhino3dm/python/api/Hatch
- PointCloud https://mcneel.github.io/rhino3dm/python/api/PointCloud
- Extrusion https://mcneel.github.io/rhino3dm/python/api/Extrusion
- Point https://mcneel.git

So, with that in mind, you could look at the docs for each type, and learn how to create them!

I've given a few more examples below, to illustrate some of the more common types and how to work with them. Notice that while it's easy to create a `Polyline` or `Line`, you will always need to add `PolylineCurve` or `LineCurve` objects. So its helpful to understand how to add those.


In [ ]:
# Line
# The line requires two 3D locatons for its start and end
test_line = rhino3dm.LineCurve(rhino3dm.Point3d(0, 0, 0), rhino3dm.Point3d(10, 10, 10))

# Polylne
# A polyline can be made from a list of points
test_polyline = rhino3dm.Polyline(
    [
        rhino3dm.Point3d(0, 0, 0),
        rhino3dm.Point3d(0, 10, 0),
        rhino3dm.Point3d(0, 10, 10),
        rhino3dm.Point3d(10, 10, 10),
    ]
).ToPolylineCurve()

# Planar polyline curve
# We create this by converting a polyline to a curve
test_polyline_curve = rhino3dm.Polyline(
    [
        rhino3dm.Point3d(0, 0, 0),
        rhino3dm.Point3d(-10, 0, 0),
        rhino3dm.Point3d(-10, -10, 0),
        rhino3dm.Point3d(0, -10, 0),
        rhino3dm.Point3d(0, 0, 0),
    ]
).ToPolylineCurve()

# Extrusion
# Finally, you can make a 3D extrusion from a polyline curve
test_extrusion = rhino3dm.Extrusion.Create(test_polyline_curve, height=20, cap=True)

And, each geometry type has helpful attributes and properties.

In [ ]:
test_polyline.IsClosed

False

Now that we've created a bunch of geometry, let's try to add them to the file. First — in case you need it — you can delete all geometry from the file.

⚠ **This code will delete any and all objects from your Rhino file!**

_You can only add geometry that is derived from `BaseGeometry`. The `Point` class is one such example. The `Point3d` class, on the other hand, is used to represent locations in space, but is not itself a geometry._



In [ ]:
ids_to_delete = [o.Attributes.Id for o in new_3dm.Objects]

print("Removing all other objects in the file.")
for id_to_delete in ids_to_delete:
    print(f"Removing object ID {id_to_delete} ...")
    new_3dm.Objects.Delete(id_to_delete)

Removing all other objects in the file.
Removing object ID fbc033e5-f3b3-4746-8b47-a6f8222f1891 ...


Now, let's add the the geometry you created previously. Again, we'll add both a geometry and attributes object each time.

In [ ]:
objects_to_add = [
    test_pt,
    test_line,
    test_polyline,
    test_polyline_curve,
    test_extrusion,
]

for object_to_add in objects_to_add:
    print(f"Adding {object_to_add.ObjectType} to file ...")
    new_3dm.Objects.Add(object_to_add, test_attributes)

Adding ObjectType.Point to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Extrusion to flie ...


And that's it! You should now have 5 shiny new geometry in the Rhino file, all on the same layer.

In [ ]:
print(f"There are {len(new_3dm.Objects)} objects in the Rhino file.")

There are 5 objects in the Rhino file.


## Saving a Rhino flie
Saving is simple! Be sure to set your version appropriately.

In [ ]:
new_3dm.Write("saved_example_rhino_file.3dm", version=7)

True